## Purpose of script:
#### Reviewing Spark's tree-based models implementation
#### Referencing Jose Portilla's "Spark and Python for Big Data with PySpark" course¶

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
# use .ml.regression for tree-based regression models
from pyspark.ml.classification import (RandomForestClassifier,
                                       DecisionTreeClassifier,
                                       GBTClassifier)

In [2]:
spark = SparkSession.builder.appName('tree_based_models').getOrCreate()

In [3]:
data = spark.read.format('libsvm').load('../Datasets/sample_libsvm_data.txt')

In [4]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [5]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [8]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [9]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [11]:
dtc_pred = dtc_model.transform(test_data)
rfc_pred = rfc_model.transform(test_data)
gbt_pred = gbt_model.transform(test_data)

In [13]:
dtc_pred.show(1)

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [28.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 1 row



In [16]:
# using Multiclass as it provide additional accuracy metrics compared to binary
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [17]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [19]:
dtc_acc = acc_eval.evaluate(dtc_pred)
rfc_acc = acc_eval.evaluate(rfc_pred)
gbt_acc = acc_eval.evaluate(gbt_pred)

In [21]:
# this is test data, so accuracy is unusually high
print(f'Decision Tree Model Accuracy: {dtc_acc}')
print(f'Random Forest Model Accuracy: {rfc_acc}')
print(f'Gradient Boost Model Accuracy: {gbt_acc}')

Decision Tree Model Accuracy: 0.8461538461538461
Random Forest Model Accuracy: 1.0
Gradient Boost Model Accuracy: 0.8461538461538461
